# Statistical Pattern Recognition - Exercise 6: Gaussian processes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
import warnings


## Define plot function 

We test it on [this example from sklearn](https://scikit-learn.org/stable/auto_examples/gaussian_process/plot_gpr_noisy_targets.html#sphx-glr-auto-examples-gaussian-process-plot-gpr-noisy-targets-py)

To plot the 95% confidence interval we look up the z-score for the standard normal distribution.
Assume a standard normal with mean 0, std 1, then for input value x=1.96 the cumulative density is 0.975.
This means 97.5% of all probability mass is to the left of x.
If we now cut off left of -1.96 and right of 1.96 we will have 95% of the probability mass inside those boundaries.

```python
# import scipy.stats
# z_score = stats.norm.ppf(0.975)  # ~1.96
```

Ppf here means percent-point function (inverse of the cumulative density function or CDF) for a standard normal distribution.


In [ ]:
def plot_gp(mu, cov, x, x_train=None, y_train=None, label="Pred"):
    """
        mu: mean prediction, shape (N, 1)
        cov: prediction covariance, shape (N, N)
        x: inputs to predict for, shape (N, 1)
        x_train: observation inputs, shape (N_obs, 1)
        y_train: obersvation labels, shape (N_obs,)
    """
    x, mu = x.ravel(), mu.ravel()  # flatten both to shape (N, )
    uncertainty = 1.96 * np.sqrt(np.diag(cov))

    plt.fill_between(x, mu + uncertainty, mu - uncertainty, alpha=0.1)
    plt.plot(x, mu, label=label)

    if x_train is not None:
        n_samples = x_train.shape[0]
        plt.plot(x_train, y_train, "rx", label=f"{n_samples} train samples")
    plt.legend()


In [ ]:
# run the gp example from sklearn
e_x = np.linspace(start=0, stop=10, num=1_000).reshape(-1, 1)
e_y = np.squeeze(e_x * np.sin(e_x))
rng = np.random.RandomState(1)
training_indices = rng.choice(np.arange(e_y.size), size=6, replace=False)
e_x_train, e_y_train = e_x[training_indices], e_y[training_indices]
kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gaussian_process = GaussianProcessRegressor(kernel=kernel, optimizer=None)
gaussian_process.fit(e_x_train, e_y_train)
print(f"{gaussian_process.kernel_=}")

# run our plot code
mu, cov = gaussian_process.predict(e_x, return_cov=True)
plot_gp(mu, cov, e_x, e_x_train, e_y_train)
plt.show()

# compare with sklearn's plot code
mean_prediction, std_prediction = gaussian_process.predict(e_x, return_std=True)
plt.plot(e_x, e_y, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
plt.scatter(e_x_train, e_y_train, label="Observations")
plt.plot(e_x, mean_prediction, label="Mean prediction")
plt.fill_between(
    e_x.ravel(),
    mean_prediction - 1.96 * std_prediction,
    mean_prediction + 1.96 * std_prediction,
    alpha=0.5,
    label=r"95% confidence interval",
)
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
plt.title("Gaussian process regression on noise-free dataset")
plt.show()


## $\star$ Part 1: Gaussian process

Load the points from regression.npz. 

Estimate the mean prediction and the variance using a Gaussian process and plot both in the style of last week’s assignment. 

Use an RBF kernel (a Gaussian function). Play
with the hyperparameters (which hyperparameters do you have?) and see
the effect in the predictive distribution. 

Reduce the number of samples and
repeat the experiments.


In [ ]:
# START TODO ################
# Load data and reshape it into X_train shape (N, 1) and Y_train shape (N,)
# Create a suitable range X_test for testing
raise NotImplementedError
# END TODO ################


In [ ]:
# START TODO ################
# Create a function that fits a GP using sklearn and run the function on the data
raise NotImplementedError
# END TODO ################


In [ ]:
# START TODO ################
# Randomly subsample your data and run the GP on [1, 2, 3, 5, 10, 15, 20] datapoints.
# What do you observe?
raise NotImplementedError
# END TODO ################


In [ ]:
# START TODO ################
# Go back to the full dataset and play with the other hyperparameters.
raise NotImplementedError
# END TODO ################


In [ ]:
# START TODO ################
# Now, implement your own rbf_kernel function and MyGPRegressor class.
raise NotImplementedError
# END TODO ################


In [ ]:
# START TODO ################
# Run your implementation on the data.
raise NotImplementedError
# END TODO ################


In [ ]:
# START TODO ################
# Play with the beta parameter of MyGPRegressor and also length and sigma_f parameters of rbf_kernel
# Observe how these *hyperparameters* change the data fit. Try to reason why you obtain these different results.
raise NotImplementedError
# END TODO ################


## $\star \star \star$ Part 2: Hyperparameter optimization

Can you optimize the hyperparameters? 

You must calculate the gradient of the log-likelihood with respect to each hyperparameter (this requires some knowledge on matrix calculus). 

Once you have the gradient you can run a gradient ascent on the hyperparameters. 

Make sure that the step size in the gradient ascent is not too big, otherwise it will not converge.

**Hint:** You can also use [JAX](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html) to compute gradients


In [ ]:
# START TODO ################
raise NotImplementedError
# END TODO ################


length = 1.
sigma_f = 1.
beta = 100.
gp = MyGPRegressorRBF(beta=beta, length=length, sigma_f=sigma_f)
gp.fit(x_train, y_train)
mu_test, sigma_test = gp.predict(x_test)

plot_gp(
    mu_test,
    sigma_test,
    x_test,
    x_train=x_train,
    y_train=y_train,
)
plt.show()


def log_likelihood_params(params):
    length_, sigma_f_ = params
    gp = MyGPRegressorRBF(beta=beta, length=length_, sigma_f=sigma_f_)
    gp.fit(x_train, y_train)
    ll = gp.log_likelihood()
    return ll

# gradients = grad(log_likelihood_params)(jnp.array([length, sigma_f]))
# print(gradients)


# # with warnings.filterwarnings("ignore"):
n_steps, report_every = 11, 1
alpha = -0.01
theta_min = 1e-4
for i in range(n_steps):
    length_grad, sigma_f_grad = grad(log_likelihood_params)(jnp.array([length, sigma_f]))
    new_length = length + alpha * length_grad
    new_sigma_f = sigma_f + alpha * sigma_f_grad

    # constrain hyperparameters to be positive
    new_length = jnp.maximum(new_length, theta_min)
    new_sigma_f = jnp.maximum(new_sigma_f, theta_min)

    if i % report_every == 0:
        ll = log_likelihood_params(jnp.array([length, sigma_f]))
        print(f"{i=} {ll=} {length_grad=} {sigma_f_grad=} {length=} {sigma_f=}")
        # print(gp.kernel_)
        gp = MyGPRegressorRBF(beta=beta, length=length, sigma_f=sigma_f)
        gp.fit(x_train, y_train)
        mu, cov = gp.predict(x_test)
        plot_gp(mu, cov, x_test, x_train=x_train, y_train=y_train)
        plt.title(f"{i=} likelihood: {ll:.3e}")
        plt.show()

    length = new_length
    sigma_f = new_sigma_f


In [ ]:



def calc_loglikelihood(X_train, Y_train, length=1, sigma_f=1, alpha = 0.05):
    """
    Returns log-marginal likelihood of theta for training data as well as the gradient
    """
    theta = [length, sigma_f]  # array containing the hyperparameters
    kernel = ConstantKernel(sigma_f, (1e-2, 1e2)) * RBF(length, (1e-2, 1e2))
    # kernel = get_rbf_kernel(length=length, sigma_f=sigma_f)

    # rbf = C(sigma_f) * RBF(length_scale=length)
    # rbf = RBF(length_scale=sigma)
    # rbf = partial(rbf_kernel, l=length, sigma_f=sigma_f)
    # rbf = get_rbf_kernel(length=length, sigma_f=sigma_f)
    gp = GaussianProcessRegressor(kernel=kernel, optimizer=None, alpha=alpha)
    # x = X[:, 0][:, np.newaxis]
    gp.fit(X_train, Y_train)

    ll, ll_grad = gp.log_marginal_likelihood(
        theta=theta, eval_gradient=True, clone_kernel=True
    )
    # ll, ll_grad = gp.log_marginal_likelihood(theta=theta, eval_gradient=True, clone_kernel=False)

    return gp, ll, ll_grad


# gradient asscent
# I'm having problems here , depending on the initial values chosen, the gradient asscent always seems to send one of the hyperparameters to 0 and then negative which causes an error
# I've tried switching up the learning rate but all that does is delay the error

# theta = [0.5, 1]
theta = np.array([1, 1])
theta_min = np.ones_like(theta) * 1e-3


# with warnings.filterwarnings("ignore"):
n_steps, report_every = 1001, 100
if True:
    for i in range(n_steps):
        alpha = 0.01
        # alpha = 0.01/i   # reduces the learning rate over time
        gp, ll, ll_grad = calc_loglikelihood(
            X_train, Y_train, length=theta[0], sigma_f=theta[1]
        )
        new_theta = theta + alpha * ll_grad
        new_theta = np.maximum(new_theta, theta_min)  # constrain hyperparameters to be positive
        if np.any(np.isnan(new_theta)):
            print(f"NaN detected, aborting. {ll=} {ll_grad=} {new_theta=}")
            break

        if i % report_every == 0:
            print(i, "log likelihood =", ll, "grad =", ll_grad, "hyperparams =", theta)
            print(gp.kernel_)
            mu, cov = gp.predict(X_test, return_cov=True)
            plot_gp(mu, cov, X_test, X_train=X_train, Y_train=Y_train)
            plt.title(f"{i=} likelihood: {ll:.3e}")
            plt.show()
        theta = new_theta


